## Trial-runs for Data Ingestion step

In [1]:
import os 

In [2]:
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\Rising-Village-Prediction-Model")

## Trial-runs for entity_config file

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path 
    unzip_dir: Path   

## Traisl-runs for ConfigurationManager file

In [4]:
#Importing dependencies 
from raisingVillage.constants import *
from raisingVillage.utils.common import read_yaml, create_directories

In [6]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Trial-runs for Components file

In [7]:
import os
from urllib import request
import zipfile
from raisingVillage import logger
from raisingVillage.utils.common import get_size


In [8]:
#Unpdate Components step
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with the following info: n\{headers}")
        else: 
            logger.info(f"File already exists: {get_size(Path(self.config.local_data_file))}")
        print("Downloding", self.config.source_url)
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts zip file into directory 
        Function returne none
        
        """
        
        unzip_path = self.config.unzip_dir
        # Print the actual path of the local data file and unzip directory
        print(f"Extracting file from: {self.config.local_data_file}")
        print(f"Unzip path: {unzip_path}")

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

## Trial-runs for pipeline 

In [9]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    data_ingestion_config =config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-27 21:50:09,576: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-05-27 21:50:09,585: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-05-27 21:50:09,597: INFO: common: yaml_file: schema.yaml loaded successfully]
[2025-05-27 21:50:09,601: INFO: common: Created directory at: artifacts]
[2025-05-27 21:50:09,605: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-27 21:50:15,599: INFO: 1819630040: artifacts/data_ingestion/data.zip download! with the following info: n\Connection: close
Content-Length: 956420
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b563e1853abc5bbe97acae5f0c2a2a20e9f5cc3a9aec3235ec24faa144b15667"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 4294:2D3213:20FE54:51674B:6835DF30
Accept-Ranges: bytes
Date: T